# Time Series Forecasting with Liquid Neural Networks

This notebook demonstrates how to use liquid neural networks (CfC and LTC) for time series forecasting. We'll cover:
- Single-step and multi-step forecasting
- Handling variable time steps
- Incorporating multiple features
- Uncertainty estimation

In [ ]:
import mlx.core as mx
import mlx.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from ncps.mlx import CfC, LTC
from sklearn.preprocessing import StandardScaler
import pandas as pd
from datetime import datetime, timedelta

## Create Forecasting Models

We'll implement models for both single-step and multi-step forecasting:

In [ ]:
class LiquidForecaster(nn.Module):
    """Time series forecasting model using liquid neurons."""
    
    def __init__(self, input_size, hidden_size, output_steps=1, cell_type='cfc'):
        super().__init__()
        self.output_steps = output_steps
        
        # Feature extraction
        self.feature_extractor = CfC if cell_type == 'cfc' else LTC
        self.feature_extractor = self.feature_extractor(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=2,
            backbone_units=64,
            backbone_layers=2,
            return_sequences=True
        )
        
        # Prediction heads
        self.forecast_head = nn.Linear(hidden_size, input_size * output_steps)
        self.uncertainty_head = nn.Linear(hidden_size, input_size * output_steps)
    
    def __call__(self, x, time_delta=None):
        # Extract features
        features = self.feature_extractor(x, time_delta=time_delta)
        # Use last state for prediction
        last_features = features[:, -1]
        
        # Generate predictions and uncertainty estimates
        forecast = self.forecast_head(last_features)
        uncertainty = nn.softplus(self.uncertainty_head(last_features))
        
        # Reshape for multi-step predictions
        batch_size = x.shape[0]
        forecast = forecast.reshape(batch_size, self.output_steps, -1)
        uncertainty = uncertainty.reshape(batch_size, self.output_steps, -1)
        
        return forecast, uncertainty

## Generate Synthetic Data

We'll create synthetic time series data with multiple seasonal components:

In [ ]:
def generate_time_series(n_samples=1000, seq_length=100, n_features=3):
    """Generate synthetic time series with multiple components."""
    # Time points
    t = np.linspace(0, 8*np.pi, seq_length)
    
    # Generate data
    data = np.zeros((n_samples, seq_length, n_features))
    targets = np.zeros((n_samples, n_features))
    
    for i in range(n_samples):
        # Generate components with different frequencies
        trend = 0.1 * t
        seasonal1 = 2 * np.sin(t + np.random.rand() * np.pi)
        seasonal2 = np.sin(2*t + np.random.rand() * np.pi)
        noise = 0.2 * np.random.randn(seq_length)
        
        # Combine components
        base_signal = trend + seasonal1 + seasonal2 + noise
        
        # Create multiple features
        for j in range(n_features):
            phase_shift = j * np.pi / 4
            amplitude = 1.0 + 0.2 * j
            data[i, :, j] = amplitude * base_signal + np.sin(t + phase_shift)
        
        # Generate target (next value)
        t_next = t[-1] + (t[1] - t[0])
        for j in range(n_features):
            phase_shift = j * np.pi / 4
            amplitude = 1.0 + 0.2 * j
            trend_next = 0.1 * t_next
            seasonal1_next = 2 * np.sin(t_next + np.random.rand() * np.pi)
            seasonal2_next = np.sin(2*t_next + np.random.rand() * np.pi)
            targets[i, j] = amplitude * (trend_next + seasonal1_next + seasonal2_next)
    
    # Generate variable time steps
    time_delta = np.ones((n_samples, seq_length, 1))
    for i in range(n_samples):
        time_delta[i] += 0.1 * np.random.randn(seq_length, 1)
    
    return mx.array(data), mx.array(targets), mx.array(time_delta)

# Generate data
X, y, time_delta = generate_time_series()

# Plot example sequence
plt.figure(figsize=(15, 5))
for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.plot(X[0, :, i], label=f'Feature {i+1}')
    plt.scatter(len(X[0]), y[0, i], c='r', label='Target')
    plt.title(f'Feature {i+1} with Target')
    plt.legend()
plt.tight_layout()
plt.show()

## Train Models

In [ ]:
def train_forecaster(model, X, y, time_delta, n_epochs=100, batch_size=32):
    """Train the forecasting model."""
    optimizer = nn.Adam(learning_rate=0.001)
    
    def loss_fn(model, x, y, dt):
        # Get predictions and uncertainty estimates
        pred, uncertainty = model(x, time_delta=dt)
        
        # Negative log likelihood loss with uncertainty
        squared_error = (pred[:, 0] - y) ** 2  # Use first prediction step
        uncertainty_term = mx.log(uncertainty[:, 0] + 1e-6)
        loss = mx.mean(squared_error / (2 * uncertainty[:, 0] + 1e-6) + uncertainty_term)
        
        return loss
    
    loss_and_grad_fn = nn.value_and_grad(model, loss_fn)
    n_samples = X.shape[0]
    losses = []
    
    for epoch in range(n_epochs):
        epoch_losses = []
        # Shuffle data
        indices = np.random.permutation(n_samples)
        
        for i in range(0, n_samples, batch_size):
            batch_idx = indices[i:i+batch_size]
            batch_X = X[batch_idx]
            batch_y = y[batch_idx]
            batch_dt = time_delta[batch_idx]
            
            loss, grads = loss_and_grad_fn(model, batch_X, batch_y, batch_dt)
            optimizer.update(model, grads)
            epoch_losses.append(float(loss))
        
        avg_loss = np.mean(epoch_losses)
        losses.append(avg_loss)
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")
    
    return losses

# Train CfC model
print("Training CfC forecaster...")
cfc_model = LiquidForecaster(input_size=3, hidden_size=32, output_steps=5, cell_type='cfc')
cfc_losses = train_forecaster(cfc_model, X, y, time_delta)

# Train LTC model
print("\nTraining LTC forecaster...")
ltc_model = LiquidForecaster(input_size=3, hidden_size=32, output_steps=5, cell_type='ltc')
ltc_losses = train_forecaster(ltc_model, X, y, time_delta)

# Plot training curves
plt.figure(figsize=(10, 5))
plt.plot(cfc_losses, label='CfC')
plt.plot(ltc_losses, label='LTC')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.grid(True)
plt.show()

## Evaluate Forecasts

In [ ]:
def evaluate_forecasts(model, X, y, time_delta):
    """Evaluate forecasting performance."""
    # Get predictions and uncertainty
    predictions, uncertainties = model(X, time_delta=time_delta)
    
    # Compute metrics
    mse = mx.mean((predictions[:, 0] - y) ** 2)
    mae = mx.mean(mx.abs(predictions[:, 0] - y))
    
    # Compute calibration (percentage of true values within uncertainty bounds)
    std = mx.sqrt(uncertainties[:, 0])
    lower_bound = predictions[:, 0] - 2 * std
    upper_bound = predictions[:, 0] + 2 * std
    calibration = mx.mean((y >= lower_bound) & (y <= upper_bound))
    
    return {
        'mse': float(mse),
        'mae': float(mae),
        'calibration': float(calibration),
        'predictions': predictions,
        'uncertainties': uncertainties
    }

# Evaluate models
cfc_results = evaluate_forecasts(cfc_model, X, y, time_delta)
ltc_results = evaluate_forecasts(ltc_model, X, y, time_delta)

print("CfC Results:")
print(f"MSE: {cfc_results['mse']:.4f}")
print(f"MAE: {cfc_results['mae']:.4f}")
print(f"Calibration: {cfc_results['calibration']:.4f}")

print("\nLTC Results:")
print(f"MSE: {ltc_results['mse']:.4f}")
print(f"MAE: {ltc_results['mae']:.4f}")
print(f"Calibration: {ltc_results['calibration']:.4f}")

# Plot example forecasts
def plot_forecast(model_name, predictions, uncertainties, true_values, feature_idx=0):
    plt.figure(figsize=(12, 5))
    
    # Plot predictions with uncertainty
    steps = np.arange(predictions.shape[1])
    mean = predictions[0, :, feature_idx]
    std = mx.sqrt(uncertainties[0, :, feature_idx])
    
    plt.plot(steps, mean, 'b-', label='Prediction')
    plt.fill_between(steps, 
                     mean - 2*std,
                     mean + 2*std,
                     color='b', alpha=0.2,
                     label='95% Confidence')
    
    # Plot true value
    plt.scatter(0, true_values[0, feature_idx], 
               c='r', label='True Value')
    
    plt.title(f'{model_name} Multi-step Forecast (Feature {feature_idx+1})')
    plt.xlabel('Steps Ahead')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot example forecasts for each model
plot_forecast('CfC', cfc_results['predictions'], 
             cfc_results['uncertainties'], y)
plot_forecast('LTC', ltc_results['predictions'], 
             ltc_results['uncertainties'], y)

## Real-World Example: Stock Price Forecasting

Let's apply our models to stock price prediction:

In [ ]:
def generate_stock_data(n_days=1000):
    """Generate realistic stock market data."""
    # Generate dates
    dates = [datetime.now() - timedelta(days=i) for i in range(n_days)]
    dates = dates[::-1]
    
    # Initial price
    price = 100.0
    prices = [price]
    volumes = []
    volatilities = []
    
    # Generate daily data
    for i in range(1, n_days):
        # Price movement
        daily_return = np.random.normal(0.0001, 0.02)
        price *= (1 + daily_return)
        prices.append(price)
        
        # Trading volume (higher on volatile days)
        base_volume = 1000000
        volume = base_volume * (1 + abs(daily_return) * 10)
        volumes.append(volume)
        
        # Volatility
        volatility = abs(daily_return)
        volatilities.append(volatility)
    
    # Create DataFrame
    df = pd.DataFrame({
        'date': dates,
        'price': prices,
        'volume': volumes,
        'volatility': volatilities
    })
    
    return df

# Generate stock data
stock_data = generate_stock_data()

# Prepare sequences
def prepare_sequences(data, seq_length=50):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data[['price', 'volume', 'volatility']])
    
    sequences = []
    targets = []
    
    for i in range(len(scaled_data) - seq_length):
        sequences.append(scaled_data[i:i+seq_length])
        targets.append(scaled_data[i+seq_length, 0])  # Predict next price
    
    return np.array(sequences), np.array(targets), scaler

# Prepare data
X_stock, y_stock, scaler = prepare_sequences(stock_data)
time_delta_stock = np.ones((X_stock.shape[0], X_stock.shape[1], 1))

# Convert to MLX arrays
X_stock = mx.array(X_stock)
y_stock = mx.array(y_stock)
time_delta_stock = mx.array(time_delta_stock)

# Train stock forecaster
stock_model = LiquidForecaster(input_size=3, hidden_size=64, output_steps=5, cell_type='cfc')
stock_losses = train_forecaster(stock_model, X_stock, y_stock, time_delta_stock)

# Evaluate stock predictions
stock_results = evaluate_forecasts(stock_model, X_stock, y_stock, time_delta_stock)

print("Stock Forecasting Results:")
print(f"MSE: {stock_results['mse']:.4f}")
print(f"MAE: {stock_results['mae']:.4f}")
print(f"Calibration: {stock_results['calibration']:.4f}")

# Plot stock predictions
plot_forecast('Stock Price', stock_results['predictions'],
             stock_results['uncertainties'], y_stock.reshape(-1, 1))